In [9]:
import bmll2 as b2
b2.get_file('modules/auxiliary_functions.py')

'auxiliary_functions.py'

In [10]:
import auxiliary_functions as af

import random
import math
import pandas as pd
import numpy as np
from pandas import StringDtype
from pathlib import Path
import shutil

# we need to add the functions because having them in the .py files only mkaes it accessible in those files so we either import it
# again or we need to do something like dh.pd.DataFrame. The dh. part means that use the module dh and from that module use pd.DataFrame

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import LogFormatterSciNotation

In [11]:
b2.get_file('top100_tickers.csv')
top100_tickers = pd.read_csv('top100_tickers.csv')

In [12]:
top100_tickers = top100_tickers['0'].tolist()

In [15]:
%%time
files = b2.list_files(path = 'top_100(Volume)')

for f in files:
    b2.get_file(f'top_100(Volume)/{f}')

CPU times: user 1min 3s, sys: 1min 14s, total: 2min 17s
Wall time: 2min 3s


In [20]:
df = pd.read_csv('NRP.csv')
df.head()
df = df.rename(columns = {'Ticker': 'RIC'})
df = df.sort_values(['DateTime', 'ExchangeSequenceNo'])
df.head()

,MIC,RIC,ListingId,Date,DateTime,ExchangeSequenceNo,Daily Volume,Daily Volatility,Trade Sign,Price,Volume,Mid-price before,Mid-price after(immediate),Mid-price after(delayed),Daily Volatility(alt)
180171,XJSE,NRP,418408226,2023-01-03,2023-01-03 09:01:07.634027,58932,560912.0,0.034471,1,10239.0,275.0,10121.5,10266.5,10121.5,0.034379
180172,XJSE,NRP,418408226,2023-01-03,2023-01-03 09:01:07.634079,58933,560912.0,0.034471,1,10254.0,1000.0,10121.5,10266.5,10266.5,0.034379
180173,XJSE,NRP,418408226,2023-01-03,2023-01-03 09:01:07.746198,59012,560912.0,0.034471,1,10238.0,423.0,10266.5,10266.5,10194.5,0.034379
180174,XJSE,NRP,418408226,2023-01-03,2023-01-03 09:02:15.121511,94108,560912.0,0.034471,1,10278.0,250.0,10194.5,10196.5,10194.5,0.034379
180175,XJSE,NRP,418408226,2023-01-03,2023-01-03 09:02:15.121527,94109,560912.0,0.034471,1,10278.0,154.0,10194.5,10196.5,10218.5,0.034379


In [ ]:
def load_stock(csv):
    df = pd.read_csv(csv, parse_dates = ['DateTime', 'Date'])
    df = df.rename(columns = {'Ticker': 'RIC'})
    df = df.sort_values(['DateTime', 'ExchangeSequenceNo'])
    return df
    
stocks = {
    ticker: load_stock(f'{ticker}.csv')
    for ticker in top100_tickers
}

In [7]:

def process_stock(stock_data, N = 4, participation_method = 'homogeneous', alpha = 2, timing_method = 'immediate', impact_method = 'all'):
    features_list = []

    for date, day_D in stock_data.groupby('Date', sort = True):
        #print(date)
        
        trades = day_D
        
        if trades.empty:
            continue

        f = af.trader_participation(N = N, method = participation_method, alpha = alpha, f_min = 1, f_max = trades.shape[0], seed = 1)

        c = af.cumulative_probs(f)
        output = af.orders(N = N, trades = trades, cumulative_probs = c)

        for n in range(N):
            trader_n_trades = trades.iloc[output[n]]

            if trader_n_trades.empty:
                continue

            trader_n_metaorders = af.metaorders(trader_n_trades)

            if len(trader_n_metaorders) == 0:
                continue

            trader_n_features = af.impact_df(trader_n_metaorders, timing_method = timing_method, impact_method = impact_method)

            features_list.append(trader_n_features)

    if features_list:
        return pd.concat(features_list, ignore_index = True)
    else:
        return pd.DataFrame()

In [ ]:
N                   = 4
trader_distribution = 'homogenous'
alpha               = 2
identifier          = f'{trader_distribution}_{N}'

In [ ]:
%%time
all_features = []

for ric, data in stocks.items():
    print(f'Processing {ric}')
    stock_features = process_stock(data, N = N, participation_method = trader_distribution, timing_method = 'immediate', impact_method = 'all')

    if not stock_features.empty:
        all_features.append(stock_features)

metaorder_data = pd.concat(all_features, ignore_index = True)

data  = metaorder_data.copy()
daily = data.groupby(['RIC', 'Date'], as_index = False).agg(daily_volume = ('daily volume', 'first'), intraday_volatility = ('intraday volatility', 'first'))

n = 20
daily[f'{n} AD volume']     = (daily.groupby(['RIC'])['daily_volume'].transform(lambda x: x.rolling(window = n, min_periods = 1).mean()))
daily[f'{n} AD volatility'] = (daily.groupby(['RIC'])['intraday_volatility'].transform(lambda x: x.rolling(window = n, min_periods = 1).mean()))

data = data.merge(daily[['RIC', 'Date', f'{n} AD volume', f'{n} AD volatility']], on = ['RIC', 'Date'], how = 'left')

metaorder_data.to_csv(f'metaorder_data_{identifier}.csv', index = False)
b2.put_file(f'metaorder_data_{identifier}.csv', 'test_data')
# takes about 19 hours to run 